In [2]:
import pandas as pd
import yfinance as yf

In [3]:
daily_sentiment = pd.read_csv("fng_data.csv")
btc_price_action = yf.Ticker("BTC-USD")
btc_price_action = btc_price_action.history("max")
btc_price_action.index = pd.to_datetime(btc_price_action.index).tz_localize(None)
btc_price_action.columns = map(str.lower, btc_price_action.columns)
del btc_price_action["dividends"]
del btc_price_action["stock splits"]
btc_price_action.head()

,open,high,low,close,volume
Date,,,,,
2014-09-17,465.864014,468.174011,452.421997,457.334015,21056800
2014-09-18,456.859985,456.859985,413.104004,424.440002,34483200
2014-09-19,424.102997,427.834991,384.532013,394.795990,37919700
2014-09-20,394.673004,423.295990,389.882996,408.903992,36863600
2014-09-21,408.084991,412.425995,393.181000,398.821014,26580100


Changing the date in btc_price_action into datetime

In [5]:
print(daily_sentiment['date'].dtype)  # This should ideally be 'object' indicating string

# Convert the column to datetime
daily_sentiment['date'] = pd.to_datetime(daily_sentiment['date'], dayfirst=True)

# Now, you can use .dt accessor
daily_sentiment['date'] = daily_sentiment['date'].dt.date

daily_sentiment.set_index('date', inplace=True)

# Drop the old index if it exists
if 'Unnamed: 0' in daily_sentiment.columns:
    daily_sentiment.drop(columns=['Unnamed: 0'], inplace=True)

# Display the DataFrame with 'date' as index and the old index dropped
display(daily_sentiment)

object


,fng_value,fng_classification
date,,
2024-04-06,75,Greed
2024-04-05,79,Extreme Greed
2024-04-04,70,Greed
2024-04-03,71,Greed
2024-04-02,79,Extreme Greed
...,...,...
2018-02-05,11,Extreme Fear
2018-02-04,24,Extreme Fear
2018-02-03,40,Fear


In [7]:
start_date = daily_sentiment.index.min()
end_date = daily_sentiment.index.max()

In [8]:
def stochastic_oscillator(df, k_length=5, k_smoothing=3, d_smoothing=3):
    df['low_min'] = df['low'].rolling(window=k_length).min()
    df['high_max'] = df['high'].rolling(window=k_length).max()
    df['stoch_k'] = ((df['close'] - df['low_min']) / (df['high_max'] - df['low_min'])) * 100
    df['stoch_k_smooth'] = df['stoch_k'].rolling(window=k_smoothing).mean()
    df['stoch_d'] = df['stoch_k_smooth'].rolling(window=d_smoothing).mean()
    df.drop(columns=['low_min', 'high_max'], inplace=True)
    return df

# Calculate Moving Averages
def calculate_ma(df, ma_periods=[5, 13]):
    for period in ma_periods:
        df[f'ma_{period}'] = df['close'].rolling(window=period).mean()
    return df

# Apply the stochastic oscillator function to your DataFrame
btc_price_action = stochastic_oscillator(btc_price_action)

# Apply the moving averages calculation to your DataFrame
btc_price_action = calculate_ma(btc_price_action)

# Display the DataFrame with Stochastic Oscillator and Moving Averages values
display(btc_price_action)

,open,high,low,close,volume,stoch_k,stoch_k_smooth,stoch_d,ma_5,ma_13
Date,,,,,,,,,,
2014-09-17,465.864014,468.174011,452.421997,457.334015,21056800,NaN,NaN,NaN,NaN,NaN
2014-09-18,456.859985,456.859985,413.104004,424.440002,34483200,NaN,NaN,NaN,NaN,NaN
2014-09-19,424.102997,427.834991,384.532013,394.795990,37919700,NaN,NaN,NaN,NaN,NaN
2014-09-20,394.673004,423.295990,389.882996,408.903992,36863600,NaN,NaN,NaN,NaN,NaN
2014-09-21,408.084991,412.425995,393.181000,398.821014,26580100,17.083525,NaN,NaN,416.859003,NaN
...,...,...,...,...,...,...,...,...,...,...
2024-04-02,69705.023438,69708.382812,64586.593750,65446.972656,50705240709,12.669050,49.072621,56.774907,69204.180469,68210.336538
2024-04-03,65446.671875,66914.320312,64559.898438,65980.812500,34488018367,20.840987,26.612041,44.373701,68421.777344,68247.984375
2024-04-04,65975.695312,69291.257812,65113.796875,68508.843750,34439527442,57.920405,30.476814,35.387159,68194.485156,68611.836839


In [9]:
btc_price_action = btc_price_action.loc[start_date:end_date]

In [10]:
btc_price_action = btc_price_action.merge(daily_sentiment, left_index=True, right_index=True, how='left')

In [11]:
btc_price_action.fillna(method='ffill', inplace=True)

C:\Users\Jehoiada Wong\AppData\Local\Temp\ipykernel_18560\710988571.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  btc_price_action.fillna(method='ffill', inplace=True)


In [12]:
btc_price_action.to_csv("final_data.csv")

In [13]:
df = pd.read_csv("final_data.csv")

In [15]:
df

,Date,open,high,low,close,volume,stoch_k,stoch_k_smooth,stoch_d,ma_5,ma_13,fng_value,fng_classification
0,2018-02-01,10237.299805,10288.799805,8812.280273,9170.540039,9959400448,11.098439,11.401036,27.762554,10516.127930,11085.433969,30.0,Fear
1,2018-02-02,9142.280273,9142.280273,7796.490234,8830.750000,12726899712,25.355039,18.686779,18.442888,9925.017969,10772.476262,15.0,Extreme Fear
2,2018-02-03,8852.120117,9430.750000,8251.629883,9174.910156,7263790080,39.263281,25.238920,18.442245,9500.719922,10585.923227,40.0,Fear
3,2018-02-04,9175.700195,9334.870117,8031.220215,8277.009766,7073549824,18.587977,27.735432,23.887044,9134.861914,10381.739333,24.0,Extreme Fear
4,2018-02-05,8270.540039,8364.839844,6756.680176,6955.270020,9285289984,5.622399,21.157886,24.710746,8481.695996,10080.729304,11.0,Extreme Fear
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2252,2024-04-02,69705.023438,69708.382812,64586.593750,65446.972656,50705240709,12.669050,49.072621,56.774907,69204.180469,68210.336538,79.0,Extreme Greed
2253,2024-04-03,65446.671875,66914.320312,64559.898438,65980.812500,34488018367,20.840987,26.612041,44.373701,68421.777344,68247.984375,71.0,Greed
2254,2024-04-04,65975.695312,69291.257812,65113.796875,68508.843750,34439527442,57.920405,30.476814,35.387159,68194.485156,68611.836839,70.0,Greed
2255,2024-04-05,68515.757812,68725.757812,66011.476562,67837.640625,33748230056,48.328632,42.363341,33.150732,67495.283594,68902.255108,79.0,Extreme Greed
